In [68]:
import numpy as np
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import *


In [4]:
text = '해보지 않으면 해낼 수 없다'

In [5]:
# 텍스트 -> 토큰화

result = text_to_word_sequence(text)
print("원문 : ",text)
print("토큰화 : ",result)

원문 :  해보지 않으면 해낼 수 없다
토큰화 :  ['해보지', '않으면', '해낼', '수', '없다']


In [7]:
docs = ['먼저 텍스트의 각 단어를 나누어 토큰화합니다.',
        '텍스트의 단어로 토큰화해야 딥러닝에서 인식됩니다.',
        '토큰화 한 결과는 딥러닝에서 사용할 수 있습니다.']

In [8]:
token = Tokenizer()
token.fit_on_texts(docs)

In [9]:
print(token.word_counts)

OrderedDict([('먼저', 1), ('텍스트의', 2), ('각', 1), ('단어를', 1), ('나누어', 1), ('토큰화합니다', 1), ('단어로', 1), ('토큰화해야', 1), ('딥러닝에서', 2), ('인식됩니다', 1), ('토큰화', 1), ('한', 1), ('결과는', 1), ('사용할', 1), ('수', 1), ('있습니다', 1)])


In [11]:
print(token.document_count) # 문장개수
print(token.word_docs) # 각 단어가 포함된 문장의 갯수
print(token.word_index) # 각 단어가 포함된 문장의 인덱스

3
defaultdict(<class 'int'>, {'텍스트의': 2, '나누어': 1, '각': 1, '단어를': 1, '먼저': 1, '토큰화합니다': 1, '딥러닝에서': 2, '인식됩니다': 1, '토큰화해야': 1, '단어로': 1, '결과는': 1, '수': 1, '한': 1, '사용할': 1, '있습니다': 1, '토큰화': 1})
{'텍스트의': 1, '딥러닝에서': 2, '먼저': 3, '각': 4, '단어를': 5, '나누어': 6, '토큰화합니다': 7, '단어로': 8, '토큰화해야': 9, '인식됩니다': 10, '토큰화': 11, '한': 12, '결과는': 13, '사용할': 14, '수': 15, '있습니다': 16}


In [15]:
docs = ['너무 재밌네요', "최고에요", '참 잘 만든 영화에요',
        '추천하고 싶은 영화입니다','한번 더 보고싶네요','글쎄요',
        '별로에요','생각보다 지루하네요','연기가 어색해요','재미없어요']
classes = np.array([1,1,1,1,1,0,0,0,0,0])

In [16]:
# 토큰화
token = Tokenizer()
token.fit_on_texts(docs)

In [19]:
print(token.word_index)
x = token.texts_to_sequences(docs)
# x에 저장된 서로 다른 단어의개수(배열길이)를 4로 통일(패딩)
paddedX = pad_sequences(x,4)
print("패딩결과 : ", paddedX)

{'너무': 1, '재밌네요': 2, '최고에요': 3, '참': 4, '잘': 5, '만든': 6, '영화에요': 7, '추천하고': 8, '싶은': 9, '영화입니다': 10, '한번': 11, '더': 12, '보고싶네요': 13, '글쎄요': 14, '별로에요': 15, '생각보다': 16, '지루하네요': 17, '연기가': 18, '어색해요': 19, '재미없어요': 20}
패딩결과 :  [[ 0  0  1  2]
 [ 0  0  0  3]
 [ 4  5  6  7]
 [ 0  8  9 10]
 [ 0 11 12 13]
 [ 0  0  0 14]
 [ 0  0  0 15]
 [ 0  0 16 17]
 [ 0  0 18 19]
 [ 0  0  0 20]]


In [21]:
#  딥러닝모델 작성
wordSize = len(token.word_index) + 1

In [ ]:
Embedding(wordSize,8,input_length=4)

In [22]:
# 딥러닝 감성분석 (긍정/부정)
model = Sequential()
model.add(Embedding(wordSize,8,input_length=4))
# Embedding(단어개수(voc),임베딩벡터크기(8차원),입력시퀀스길이))
# 21차원 -> 임베딩(단어>밀집벡터) -> 8차원
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
model.fit(paddedX,classes,epochs=20)
print("정확도 : %.4f" % (model.evaluate(paddedX, classes)[1]))
model.summary()

"""
단어 종류 : 21가지-> 벡터 공간 -> 21차원 -> 임베딩 => 2차원
100000...0000 => 1.5, 3.7
100000...0001 => 1.0, 2.7
임베딩 : 단어 -> 밀집 벡터 (Dense vector ))
신경말
ex) 원핫인코딩 단어 벡터의 차원이 50000 차원 => 임베딩 => 2차원
"""

In [37]:
# LSTM을 이용한 뉴스분류기

from keras.datasets import reuters
from keras.utils import np_utils

(xTrain, yTrain), (xTest, yTest) = reuters.load_data(num_words=1000, test_split= 0.2)

In [38]:
category=np.max(yTrain)+1 #뉴스 카테고리(46가지), 0~45 => 1~46
print("카테고리:", category)
print("학습데이터:", len(xTrain)) #8982 건
print("테스트데이터:", len(xTest)) #2246 건
print(xTrain[1]) #길이 87인 뉴스 기사
print(yTrain[0]) #3 카테고리 뉴스

카테고리: 46
학습데이터: 8982
테스트데이터: 2246
[1, 2, 699, 2, 2, 56, 2, 2, 9, 56, 2, 2, 81, 5, 2, 57, 366, 737, 132, 20, 2, 7, 2, 49, 2, 2, 2, 2, 699, 2, 8, 7, 10, 241, 16, 855, 129, 231, 783, 5, 4, 587, 2, 2, 2, 775, 7, 48, 34, 191, 44, 35, 2, 505, 17, 12]
3


In [39]:
xTrain=pad_sequences(xTrain,maxlen=100)
xTest=pad_sequences(xTest,maxlen=100)
yTrain=np_utils.to_categorical(yTrain)
yTest=np_utils.to_categorical(yTest)

In [40]:
#모델 생성
model=Sequential()
model.add(Embedding(1000, 100))
model.add(LSTM(100, activation='tanh'))
model.add(Dense(46, activation='softmax'))

In [41]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [42]:
history=model.fit(xTrain, yTrain, batch_size=100, epochs=20,
         validation_data=(xTest, yTest))

Epoch 1/20
90/90 [==============================] - 8s 91ms/step - loss: 2.5978 - accuracy: 0.3477 - val_loss: 2.4011 - val_accuracy: 0.3620
Epoch 2/20
90/90 [==============================] - 8s 90ms/step - loss: 2.1173 - accuracy: 0.4637 - val_loss: 2.0200 - val_accuracy: 0.4982
Epoch 3/20
90/90 [==============================] - 8s 91ms/step - loss: 1.9110 - accuracy: 0.5184 - val_loss: 1.8319 - val_accuracy: 0.5490
Epoch 4/20
90/90 [==============================] - 8s 85ms/step - loss: 1.7411 - accuracy: 0.5561 - val_loss: 1.7201 - val_accuracy: 0.5637
Epoch 5/20
90/90 [==============================] - 8s 85ms/step - loss: 1.6549 - accuracy: 0.5856 - val_loss: 1.6539 - val_accuracy: 0.5819
Epoch 6/20
90/90 [==============================] - 8s 84ms/step - loss: 1.5927 - accuracy: 0.5973 - val_loss: 1.6180 - val_accuracy: 0.5841
Epoch 7/20
90/90 [==============================] - 7s 81ms/step - loss: 1.4751 - accuracy: 0.6304 - val_loss: 1.5749 - val_accuracy: 0.6153
Epoch 8/20
90

In [ ]:
"""
LSTM : 긴 시퀀스 기억 계층
ex) LSTM(3, input_dim=1)
        출력    입력속성수
    LSTM(3, input_dim=1, input_length=4)
input_length : 시퀀스데이터의 길이
시퀀스 출력여부(True:매 입력출력, False:맨마지막에만 출력))
"""

In [44]:
# hihell 까지 넣으면 hihello 출력하는 모델 만들기

sample = 'hihello'
charSet = list(set(sample))

In [46]:
charDic = {w:i for i,w in enumerate(charSet)}
charDic

{'e': 0, 'o': 1, 'h': 2, 'i': 3, 'l': 4}

In [47]:
xStr = sample[:-1]
yStr = sample[1:]

In [53]:
datadim = len(charSet) #VOC 출력결과로 나와야 하는 문자의 종류의 수
numClasses = len(charSet) #5가지 문자 종류

In [54]:
x = [charDic[c] for c in xStr]
x # hihell의 인덱스 [2, 3, 2, 0, 4, 4]
y = [charDic[c] for c in yStr]
y # ihello의 인덱스 [3, 2, 0, 4, 4, 1]

[3, 2, 0, 4, 4, 1]

In [55]:
# 원핫인코딩
x = np_utils.to_categorical(x, num_classes=numClasses)
print(x)
print(x.shape)
y = np_utils.to_categorical(y, num_classes=numClasses)
print(y)
print(y.shape)

[[0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]]
(6, 5)
[[0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]]
(6, 5)


In [56]:
# x.shape # (6,5) 6:입력길이, 5:차원
x = np.reshape(x,(-1, len(x), datadim))
x.shape

(1, 6, 5)

In [57]:
y = np.reshape(y,(-1, len(y), datadim))
y.shape

(1, 6, 5)

In [64]:
# 모델 생성
model = Sequential()

# 좋지 않은 모델 구조
# model.add(LSTM( 5, # 출력되어야할 문자의 종류
#                 input_shape=(6,5), # timesteps(6글자입력) / dataDim(입력문자차원(종류))
#                 return_sequences=True))

# 계층을 딥하고 와이드하게 설계
model.add(LSTM( 5*128, input_shape=(6,5), return_sequences=True ))
model.add(LSTM( 5*64, return_sequences=True ))
model.add(LSTM( 5*16, return_sequences=True ))
model.add(LSTM( 5*4, return_sequences=True ))
model.add(Dense( 5 ))
model.add(Activation('softmax')) #여러개의 문자중에 하나 출력 : softmax
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 6, 640)            1653760   
_________________________________________________________________
lstm_4 (LSTM)                (None, 6, 320)            1230080   
_________________________________________________________________
lstm_5 (LSTM)                (None, 6, 80)             128320    
_________________________________________________________________
lstm_6 (LSTM)                (None, 6, 20)             8080      
_________________________________________________________________
dense_2 (Dense)              (None, 6, 5)              105       
_________________________________________________________________
activation_1 (Activation)    (None, 6, 5)              0         
Total params: 3,020,345
Trainable params: 3,020,345
Non-trainable params: 0
____________________________________________

In [65]:
model.compile(loss= 'categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
model.fit(x,y,epochs=200)

Epoch 1/200
1/1 [==============================] - 0s 1ms/step - loss: 1.6103 - accuracy: 0.1667
Epoch 2/200
1/1 [==============================] - 0s 1ms/step - loss: 1.5992 - accuracy: 0.5000
Epoch 3/200
1/1 [==============================] - 0s 1ms/step - loss: 1.5862 - accuracy: 0.5000
Epoch 4/200
1/1 [==============================] - 0s 1ms/step - loss: 1.5678 - accuracy: 0.5000
Epoch 5/200
1/1 [==============================] - 0s 1ms/step - loss: 1.5408 - accuracy: 0.3333
Epoch 6/200
1/1 [==============================] - 0s 2ms/step - loss: 1.5032 - accuracy: 0.1667
Epoch 7/200
1/1 [==============================] - 0s 2ms/step - loss: 1.4585 - accuracy: 0.1667
Epoch 8/200
1/1 [==============================] - 0s 2ms/step - loss: 1.4174 - accuracy: 0.1667
Epoch 9/200
1/1 [==============================] - 0s 2ms/step - loss: 1.3875 - accuracy: 0.1667
Epoch 10/200
1/1 [==============================] - 0s 1ms/step - loss: 1.3675 - accuracy: 0.1667
Epoch 11/200
1/1 [===========

In [66]:
predictions = model.predict(x)
predictions

array([[[3.1467823e-03, 3.3309341e-03, 3.9268788e-02, 9.4730985e-01,
         6.9435807e-03],
        [9.0610487e-03, 7.8992610e-04, 9.5688099e-01, 2.9319841e-02,
         3.9481181e-03],
        [9.7389054e-01, 1.3075087e-03, 7.9952935e-03, 1.0488108e-03,
         1.5757930e-02],
        [1.2912435e-02, 3.0295732e-03, 3.4067556e-03, 8.9128432e-04,
         9.7975999e-01],
        [5.0189905e-03, 1.6118433e-02, 1.3648822e-03, 1.1943213e-03,
         9.7630346e-01],
        [1.0110065e-03, 9.7210127e-01, 1.1962255e-03, 6.8613573e-04,
         2.5005359e-02]]], dtype=float32)

In [67]:
for i, pred in enumerate(predictions):
    xindex = np.argmax(x[i], axis=1)
    xstr = [charSet[j] for j in xindex]
    print(xindex,"".join(xstr))

    ind = np.argmax(pred, axis=1)
    res = [charSet[j] for j in ind]
    print(ind,''.join(res))

"""
첫번째 모델 결과 :
[2 3 2 0 4 4] hihell
[0 0 0 1 1 1] eeeooo
좋은 모델이 아님!
"""

"""
딥하게 구조한 모델 결과 :
[2 3 2 0 4 4] hihell
[3 2 0 4 4 1] ihello
정확하게 예측함
"""

[2 3 2 0 4 4] hihell
[3 2 0 4 4 1] ihello


'\n결과 :\n[2 3 2 0 4 4] hihell\n[0 0 0 1 1 1] eeeooo\n좋은 모델이 아님!\n'